In [ ]:
# Instalamos pyspark y py4j
!pip install pyspark py4j

In [201]:
from pyspark.sql import SparkSession

Spark = SparkSession.builder.appName("lion").getOrCreate()
sc = Spark.sparkContext

# **EJERCICIOS / PRACTICA**

In [ ]:
# 1. Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, #R, C, Scala, Rugby y SQL.
#   A) Cree un nuevo RDD que solo contenga aquellos lenguajes de programación que comiencen con la letra R.

# 2. Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].
#   B) Obtenga una lista compuesta por los números pares en el intervalo
#      [20;30] y sus respectivas raíces cuadradas. Un ejemplo del resultado deseado para
#      el intervalo [50;60] sería la lista [50, 7.0710678118654755, 52, 7.21110255092 54,
#      7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834

#   C) Eleve el número de particiones del RDD sqrt a 20.

#   D) Si tuviera que disminuir el número de particiones luego de haberlo
#      establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?

# 3. Cree un RDD del tipo clave valor a partir de la lectura del TXT de transacciones.
#    Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado.
#    Supongamos que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas.
#    Obtenga el monto total por cada cuenta. (Suma)

# Tip: Cree su propia función para procesar el RDD leído.

# **1.- Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, #R, C, Scala, Rugby y SQL.**

In [175]:
rdd_lenguajes = sc.parallelize(["Python", "R", "C", "Scala", "Rugby", "SQL"])
rdd_lenguajes.collect()

['Python', 'R', 'C', 'Scala', 'Rugby', 'SQL']

### A) Cree un nuevo RDD que solo contenga aquellos lenguajes de programación que comiencen con la letra R.


In [176]:
rdd_lenguajes_con_r = rdd_lenguajes.filter(lambda x: x.startswith("R"))
rdd_lenguajes_con_r.collect()

['R', 'Rugby']

#**2. Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].**

In [ ]:
numeros = list(range(20, 30))
rdd_numeros = sc.parallelize(numeros)

rdd_pares = rdd_numeros.filter(lambda x: x % 2 == 0)

rdd_pares.collect()

[20, 22, 24, 26, 28]

## B) Obtenga una lista compuesta por los números pares en el intervalo [20;30] y sus respectivas raíces cuadradas.

> Un ejemplo del resultado deseado para el intervalo [50;60] sería la lista: [50, 7.0710678118654755, 52, 7.21110255092 54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834.

In [ ]:
rdd_raices_flat = rdd_pares.flatMap(lambda x: [x, x ** 0.5])
rdd_raices_flat.collect()

[20,
 4.47213595499958,
 22,
 4.69041575982343,
 24,
 4.898979485566356,
 26,
 5.0990195135927845,
 28,
 5.291502622129181]

##   C) Eleve el número de particiones del RDD sqrt a 20.

In [ ]:
rdd_elevar = rdd_raices_flat.repartition(20)
rdd_elevar.getNumPartitions()

20

## D) Si tuviera que disminuir el número de particiones luego de haberlo establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?


In [ ]:
#   RESPUESTA: Para disminuir el numero de particiones luego de haberlo establecido a 20 usaria
#              la función COALESCE ya que esta función es mas practica para disminuir el número
#              de particiones porque tambien evita el movimiento completo de datos entre las particiones.

rdd_disminuir = rdd_elevar.coalesce(5)
rdd_disminuir.getNumPartitions()

5

# **3. Cree un RDD del tipo clave valor a partir de la lectura del TXT de transacciones.:**
> Lo realice de 2 Maneras, la primera es la normal y copiando e creando el RDD en clave valor y la segunda leyendo el documento transacciones.txt con funcion de imprimir print y mostrando algo mas limpio con texto.




1.   Tenga en cuenta que deberá procesar el RDD leído para obtener  el resultado solicitado.
2.   Supongamos que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas.
3.   Obtenga el monto total por cada cuenta. (Suma)
4.   **Tip:** Cree su propia función para procesar el RDD leído.


## **MANERA 1**

In [204]:
# Datos del TXT transacciones
datos_transtxt = [
  (1001, 52.3),
  (1005, 20.8),
  (1001, 10.1),
  (1004, 52.7),
  (1005, 20.7),
  (1002, 85.3),
  (1004, 20.9)
]

# Convierto los datos en un RDD
rdd_trant = sc.parallelize(datos_transtxt)

# Hago que se calcule el monto total por cada cuenta utilizando reduceByKey
monto_total_por_cuenta = rdd_trant.reduceByKey(lambda x, y: x + y)

monto_total_por_cuenta.collect()


[(1004, 73.6), (1002, 85.3), (1001, 62.4), (1005, 41.5)]

## **MANERA 2**



In [202]:
leer = '/content/transacciones.txt'
rdd_lineas = sc.textFile(leer)

rdd_mostrar = rdd_lineas.map(lambda line: ((line.split()[0], line.split()[1])))

rdd_mostrar.collect()

[('(1001,', '52.3)'),
 ('(1005,', '20.8)'),
 ('(1001,', '10.1)'),
 ('(1004,', '52.7)'),
 ('(1005,', '20.7)'),
 ('(1002,', '85.3)'),
 ('(1004,', '20.9)')]

In [203]:
# Función para extraer números con variable llamada "extraer_numeros" de una cadena y convertirlos en floats.
def extraer_numeros(line):
    # Creo una variable llamada "valores" para poder borrar y remplazar los paréntesis, comas,guiones y los dividir por coma.
    valores = line.replace('(', '').replace(')', '').replace('-', '').split(',')
    try:
    # Convierto el segundo valor a float y devolverlo.
        return (valores[0], float(valores[1]))
    except Error:
        return None

# Filtrar las líneas que no contienen números con la variable de "extraer_numeros" y enviarlo limpio de simbolos en la varia.
rdd_numeros = rdd_lineas.map(extraer_numeros).filter(lambda x: x is not None)

# Reducir por clave para sumar los valores por la misma clave.
rdd_resultado = rdd_numeros.reduceByKey(lambda x, y: x + y).collect()

# Imprimo el resultado de forma ordenada y limpia sin comas ni simbolos en el resultado.
for clave, suma in rdd_resultado:
    print(f"Cuenta: {clave} - Suma de cada cuenta: {suma}")

Cuenta: 1002 - Suma de cada cuenta: 85.3
Cuenta: 1001 - Suma de cada cuenta: 62.4
Cuenta: 1005 - Suma de cada cuenta: 41.5
Cuenta: 1004 - Suma de cada cuenta: 73.6
